# Importujemy biblioteki

In [2]:
# Importujemy biblioteki
import pandas as pd
import numpy as np
import os
import ast
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pickle

Reszta kodu dla proporcji 70_30 PNN

In [ ]:
# Ścieżki do danych
train_texts_path = '../data/70_30/train_texts_w2v.csv'
train_labels_path = '../data/70_30/train_labels.csv'

# Wczytujemy dane
X_train_raw = pd.read_csv(train_texts_path, index_col=0)
y_train = pd.read_csv(train_labels_path, index_col=0).squeeze()

print("Dane treningowe wczytane poprawnie.")
print(f"X_train_raw shape: {X_train_raw.shape}")
print(f"y_train shape: {y_train.shape}")

print("Rozkład klas w zbiorze treningowym (przed mapowaniem):")
print(y_train.value_counts(normalize=True))

# Wyrównujemy indeksy
X_train_raw, y_train = X_train_raw.align(y_train, join='inner', axis=0)

# Mapujemy gwiazdki na klasy sentymentu
def map_sentiment(star_rating):
    if star_rating in [4, 5]:
        return 'Pozytywna'
    elif star_rating == 3:
        return 'Neutralna'
    else:
        return 'Negatywna'

y_train = y_train.map(map_sentiment)

print("Rozkład klas w zbiorze treningowym (po mapowaniu):")
print(y_train.value_counts(normalize=True))

# Parsujemy wektory Word2Vec
X_train = X_train_raw['text'].apply(ast.literal_eval).apply(np.array)
X_train = np.stack(X_train.values)

print(f"Kształt danych wejściowych po konwersji: {X_train.shape}")

# Kodujemy etykiety
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_train_categorical = to_categorical(y_train_encoded)

print(f"Zakodowane etykiety: {list(label_encoder.classes_)}")

# Parametry wejściowe
input_shape = (X_train.shape[1], X_train.shape[2])

# Budujemy model LSTM
model = Sequential()
model.add(Masking(mask_value=0., input_shape=input_shape))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train_categorical.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

# Trenowanie modelu
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

model.fit(X_train, y_train_categorical, epochs=10, batch_size=128, callbacks=[early_stopping])

print("Model został wytrenowany.")

# Zapisujemy model do pliku .h5
model_dir = '../models'
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, 'w2v_LSTM_70_30_PNN.h5')
model.save(model_path)

print(f"Model zapisano w: {model_path}")

# Zapisujemy label encoder do późniejszego dekodowania
with open(os.path.join(model_dir, 'w2v_LSTM_70_30_PNN_label_encoder.pkl'), 'wb') as le_file:
    pickle.dump(label_encoder, le_file)

print("LabelEncoder został zapisany.")


Dane treningowe wczytane poprawnie.
X_train_raw shape: (69997, 1)
y_train shape: (70000,)
Rozkład klas w zbiorze treningowym (przed mapowaniem):
stars
5    0.443514
4    0.253043
3    0.113557
1    0.109243
2    0.080643
Name: proportion, dtype: float64
Rozkład klas w zbiorze treningowym (po mapowaniu):
stars
Pozytywna    0.696557
Negatywna    0.189886
Neutralna    0.113557
Name: proportion, dtype: float64
Po align: X_train_raw shape: (69997, 1), y_train shape: (69997,)
Kształt danych wejściowych po paddingu: (69997, 500, 50)
Maksymalna długość sekwencji po paddingu: 500
Wymiar wektora embeddingów: 50
Zakodowane etykiety: ['Negatywna', 'Neutralna', 'Pozytywna']


c:\Users\Maciej\Desktop\sem6\UCZENIE MASZYNOWE\sentiment_analysis\.venv\lib\site-packages\keras\src\layers\core\masking.py:48: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 500, 50)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        29,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,619 (123.51 KB)

 Trainable params: 31,619 (123.51 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 250s 452ms/step - accuracy: 0.7718 - loss: 0.5720
Epoch 2/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 251s 459ms/step - accuracy: 0.8433 - loss: 0.3988
Epoch 3/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 261s 477ms/step - accuracy: 0.8571 - loss: 0.3623
Epoch 4/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 262s 479ms/step - accuracy: 0.8658 - loss: 0.3419
Epoch 5/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 266s 486ms/step - accuracy: 0.8696 - loss: 0.3296
Epoch 6/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 272s 497ms/step - accuracy: 0.8734 - loss: 0.3179
Epoch 7/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 270s 493ms/step - accuracy: 0.8801 - loss: 0.3023
Epoch 8/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 273s 499ms/step - accuracy: 0.8833 - loss: 0.2926
Epoch 9/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 272s 497ms/step - accuracy: 0.8877 - loss: 0.2824
Epoch 10/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 264s 482ms/step - accuracy: 0.8893 - loss: 0.2754


Model został wytrenowany.
Model zapisano w: ../models\w2v_LSTM_70_30_PNN.h5
LabelEncoder został zapisany.


dla propocji 80-20 PNN

In [8]:
# Ścieżki do danych
train_texts_path = '../data/80_20/train_texts_w2v.csv'
train_labels_path = '../data/80_20/train_labels.csv'

# Wczytujemy dane
X_train_raw = pd.read_csv(train_texts_path, index_col=0)
y_train = pd.read_csv(train_labels_path, index_col=0).squeeze()

print("Dane treningowe wczytane poprawnie.")
print(f"X_train_raw shape: {X_train_raw.shape}")
print(f"y_train shape: {y_train.shape}")

print("Rozkład klas w zbiorze treningowym (przed mapowaniem):")
print(y_train.value_counts(normalize=True))

# Mapujemy gwiazdki na klasy sentymentu
def map_sentiment(star_rating):
    if star_rating in [4, 5]:
        return 'Pozytywna'
    elif star_rating == 3:
        return 'Neutralna'
    else:
        return 'Negatywna'

y_train = y_train.map(map_sentiment)

print("Rozkład klas w zbiorze treningowym (po mapowaniu):")
print(y_train.value_counts(normalize=True))

# Wyrównujemy indeksy
X_train_raw, y_train = X_train_raw.align(y_train, join='inner', axis=0)
print(f"Po align: X_train_raw shape: {X_train_raw.shape}, y_train shape: {y_train.shape}")

# Parsujemy wektory Word2Vec
X_train_list = X_train_raw['text'].apply(ast.literal_eval).apply(np.array).tolist()

# Padding sekwencji do tej samej długości
X_train_padded = pad_sequences(X_train_list, padding='post', dtype='float32')

print(f"Kształt danych wejściowych po paddingu: {X_train_padded.shape}")
print(f"Maksymalna długość sekwencji po paddingu: {X_train_padded.shape[1]}")
print(f"Wymiar wektora embeddingów: {X_train_padded.shape[2]}")

# Kodujemy etykiety
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_train_categorical = to_categorical(y_train_encoded)

print(f"Zakodowane etykiety: {list(label_encoder.classes_)}")

# Parametry wejściowe
input_shape = (X_train_padded.shape[1], X_train_padded.shape[2])

# Budujemy model LSTM
model = Sequential()
model.add(Masking(mask_value=0., input_shape=input_shape))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train_categorical.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

# Trenowanie modelu
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

model.fit(X_train_padded, y_train_categorical, epochs=10, batch_size=128, callbacks=[early_stopping])

print("Model został wytrenowany.")

# Zapisujemy model do pliku .h5
model_dir = '../models'
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, 'w2v_LSTM_80_20_PNN.h5')
model.save(model_path)

print(f"Model zapisano w: {model_path}")

# Zapisujemy label encoder do późniejszego dekodowania
with open(os.path.join(model_dir, 'w2v_LSTM_80_20_PNN_label_encoder.pkl'), 'wb') as le_file:
    pickle.dump(label_encoder, le_file)

print("LabelEncoder został zapisany.")

Dane treningowe wczytane poprawnie.
X_train_raw shape: (79997, 1)
y_train shape: (80000,)
Rozkład klas w zbiorze treningowym (przed mapowaniem):
stars
5    0.444012
4    0.253063
3    0.113587
1    0.109137
2    0.080200
Name: proportion, dtype: float64
Rozkład klas w zbiorze treningowym (po mapowaniu):
stars
Pozytywna    0.697075
Negatywna    0.189337
Neutralna    0.113587
Name: proportion, dtype: float64
Po align: X_train_raw shape: (79997, 1), y_train shape: (79997,)
Kształt danych wejściowych po paddingu: (79997, 500, 50)
Maksymalna długość sekwencji po paddingu: 500
Wymiar wektora embeddingów: 50
Zakodowane etykiety: ['Negatywna', 'Neutralna', 'Pozytywna']


c:\Users\Maciej\Desktop\sem6\UCZENIE MASZYNOWE\sentiment_analysis\.venv\lib\site-packages\keras\src\layers\core\masking.py:48: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_3 (Masking)             │ (None, 500, 50)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        29,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,619 (123.51 KB)

 Trainable params: 31,619 (123.51 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 388s 614ms/step - accuracy: 0.7894 - loss: 0.5430
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 370s 591ms/step - accuracy: 0.8476 - loss: 0.3863
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 373s 596ms/step - accuracy: 0.8581 - loss: 0.3584
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 360s 576ms/step - accuracy: 0.8688 - loss: 0.3341
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 368s 588ms/step - accuracy: 0.8707 - loss: 0.3253
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 363s 581ms/step - accuracy: 0.8761 - loss: 0.3124
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 368s 588ms/step - accuracy: 0.8806 - loss: 0.3000
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 371s 593ms/step - accuracy: 0.8868 - loss: 0.2857
Epoch 9/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 372s 595ms/step - accuracy: 0.8896 - loss: 0.2807
Epoch 10/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 360s 577ms/step - accuracy: 0.8933 - loss: 0.2699


Model został wytrenowany.
Model zapisano w: ../models\w2v_LSTM_80_20_PNN.h5
LabelEncoder został zapisany.


Analiza LSTM dla 5 klas (gwiazdek) w proporcji 70-30

In [4]:
# Ścieżki do danych
train_texts_path = '../data/70_30/train_texts_w2v.csv'
train_labels_path = '../data/70_30/train_labels.csv'

# Wczytujemy dane
X_train_raw = pd.read_csv(train_texts_path, index_col=0)
y_train = pd.read_csv(train_labels_path, index_col=0).squeeze()

print("Dane treningowe wczytane poprawnie.")
print(f"X_train_raw shape: {X_train_raw.shape}")
print(f"y_train shape: {y_train.shape}")

print("Rozkład klas w zbiorze treningowym (liczba gwiazdek):")
print(y_train.value_counts(normalize=True))

# Wyrównujemy indeksy
X_train_raw, y_train = X_train_raw.align(y_train, join='inner', axis=0)

# Parsujemy wektory Word2Vec i stosujemy padding
X_train_sequences = X_train_raw['text'].apply(ast.literal_eval).apply(np.array).tolist()
X_train_padded = pad_sequences(X_train_sequences, padding='post', dtype='float32')

print(f"Kształt danych wejściowych po paddingu: {X_train_padded.shape}")

X_train = X_train_padded

# Kodujemy etykiety
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_train_categorical = to_categorical(y_train_encoded)

print(f"Zakodowane klasy: {list(label_encoder.classes_)}")

# Parametry wejściowe
input_shape = (X_train.shape[1], X_train.shape[2])

# Budujemy model LSTM
model = Sequential()
model.add(Masking(mask_value=0., input_shape=input_shape))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train_categorical.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

# Trenowanie modelu
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

model.fit(X_train, y_train_categorical, epochs=10, batch_size=128, callbacks=[early_stopping])

print("Model został wytrenowany.")

# Zapisujemy model do pliku .h5
model_dir = '../models'
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, 'w2v_LSTM_70_30_stars.h5')
model.save(model_path)

print(f"Model zapisano w: {model_path}")

# Zapisujemy label encoder do późniejszego dekodowania
with open(os.path.join(model_dir, 'w2v_LSTM_70_30_stars_label_encoder.pkl'), 'wb') as le_file:
    pickle.dump(label_encoder, le_file)

print("LabelEncoder został zapisany.")

Dane treningowe wczytane poprawnie.
X_train_raw shape: (69997, 1)
y_train shape: (70000,)
Rozkład klas w zbiorze treningowym (liczba gwiazdek):
stars
5    0.443514
4    0.253043
3    0.113557
1    0.109243
2    0.080643
Name: proportion, dtype: float64
Kształt danych wejściowych po paddingu: (69997, 500, 50)
Zakodowane klasy: [1, 2, 3, 4, 5]


c:\Users\Maciej\Desktop\sem6\UCZENIE MASZYNOWE\sentiment_analysis\.venv\lib\site-packages\keras\src\layers\core\masking.py:48: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 500, 50)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        29,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,685 (123.77 KB)

 Trainable params: 31,685 (123.77 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 233s 423ms/step - accuracy: 0.5494 - loss: 1.0745
Epoch 2/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 251s 459ms/step - accuracy: 0.6329 - loss: 0.8499
Epoch 3/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 263s 480ms/step - accuracy: 0.6534 - loss: 0.8106
Epoch 4/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 268s 491ms/step - accuracy: 0.6607 - loss: 0.7873
Epoch 5/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 271s 496ms/step - accuracy: 0.6691 - loss: 0.7689
Epoch 6/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 271s 495ms/step - accuracy: 0.6755 - loss: 0.7546
Epoch 7/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 271s 496ms/step - accuracy: 0.6829 - loss: 0.7341
Epoch 8/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 275s 503ms/step - accuracy: 0.6886 - loss: 0.7233
Epoch 9/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 274s 500ms/step - accuracy: 0.6901 - loss: 0.7118
Epoch 10/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 272s 498ms/step - accuracy: 0.6989 - loss: 0.6974


Model został wytrenowany.
Model zapisano w: ../models\w2v_LSTM_70_30_stars.h5
LabelEncoder został zapisany.


Trenowanie Modelu dla proporcji 80-20 : system 5 gwiazdek

In [7]:
# Ścieżki do danych
train_texts_path = '../data/80_20/train_texts_w2v.csv'
train_labels_path = '../data/80_20/train_labels.csv'

# Wczytujemy dane
X_train_raw = pd.read_csv(train_texts_path, index_col=0)
y_train = pd.read_csv(train_labels_path, index_col=0).squeeze()

print("Dane treningowe wczytane poprawnie.")
print(f"X_train_raw shape: {X_train_raw.shape}")
print(f"y_train shape: {y_train.shape}")

print("Rozkład klas w zbiorze treningowym (liczba gwiazdek):")
print(y_train.value_counts(normalize=True))

# Wyrównujemy indeksy
X_train_raw, y_train = X_train_raw.align(y_train, join='inner', axis=0)

# Parsujemy wektory Word2Vec i stosujemy padding
X_train_sequences = X_train_raw['text'].apply(ast.literal_eval).apply(np.array).tolist()
X_train_padded = pad_sequences(X_train_sequences, padding='post', dtype='float32')

print(f"Kształt danych wejściowych po paddingu: {X_train_padded.shape}")

X_train = X_train_padded

# Kodujemy etykiety
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_train_categorical = to_categorical(y_train_encoded)

print(f"Zakodowane klasy: {list(label_encoder.classes_)}")

# Parametry wejściowe
input_shape = (X_train.shape[1], X_train.shape[2])

# Budujemy model LSTM
model = Sequential()
model.add(Masking(mask_value=0., input_shape=input_shape))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train_categorical.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

# Trenowanie modelu
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

model.fit(X_train, y_train_categorical, epochs=10, batch_size=128, callbacks=[early_stopping])

print("Model został wytrenowany.")

# Zapisujemy model do pliku .h5
model_dir = '../models'
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, 'w2v_LSTM_80_20_stars.h5')
model.save(model_path)

print(f"Model zapisano w: {model_path}")

# Zapisujemy label encoder do późniejszego dekodowania
with open(os.path.join(model_dir, 'w2v_LSTM_80_20_stars_label_encoder.pkl'), 'wb') as le_file:
    pickle.dump(label_encoder, le_file)

print("LabelEncoder został zapisany.")

Dane treningowe wczytane poprawnie.
X_train_raw shape: (79997, 1)
y_train shape: (80000,)
Rozkład klas w zbiorze treningowym (liczba gwiazdek):
stars
5    0.444012
4    0.253063
3    0.113587
1    0.109137
2    0.080200
Name: proportion, dtype: float64
Kształt danych wejściowych po paddingu: (79997, 500, 50)
Zakodowane klasy: [1, 2, 3, 4, 5]


c:\Users\Maciej\Desktop\sem6\UCZENIE MASZYNOWE\sentiment_analysis\.venv\lib\site-packages\keras\src\layers\core\masking.py:48: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_2 (Masking)             │ (None, 500, 50)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        29,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,685 (123.77 KB)

 Trainable params: 31,685 (123.77 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 388s 615ms/step - accuracy: 0.5415 - loss: 1.0716
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 377s 603ms/step - accuracy: 0.6386 - loss: 0.8369
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 362s 580ms/step - accuracy: 0.6506 - loss: 0.8092
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 356s 569ms/step - accuracy: 0.6597 - loss: 0.7811
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 355s 568ms/step - accuracy: 0.6681 - loss: 0.7645
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 353s 565ms/step - accuracy: 0.6762 - loss: 0.7520
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 351s 562ms/step - accuracy: 0.6809 - loss: 0.7391
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 349s 559ms/step - accuracy: 0.6872 - loss: 0.7236
Epoch 9/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 349s 558ms/step - accuracy: 0.6919 - loss: 0.7095
Epoch 10/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 350s 560ms/step - accuracy: 0.6948 - loss: 0.7058


Model został wytrenowany.
Model zapisano w: ../models\w2v_LSTM_80_20_stars.h5
LabelEncoder został zapisany.
